# Goal of this file: Go over an example of using the package Multiagent_GNN_Policies by GRASP.

In [4]:
%matplotlib inline

In [1]:
# Import packages used in train.py and gnn_cloning.py
from os import path
import configparser
import numpy as np
import random
import gym
import gym_flock
import torch
import sys
import torch.nn.functional as F
from torch.optim import Adam
from torch.autograd import Variable
from multiagent_gnn_policies.learner.actor import Actor

from multiagent_gnn_policies.learner.gnn_cloning import train_cloning

AirSim not installed.


In [ ]:
# Define an actor that doesn't 
class Actor(nn.Module):

    def __init__(self, n_s, n_a, hidden_layers, k, ind_agg):
        """
        The policy network is allowed to have only one aggregation operation due to communication latency, but we can
        have any number of hidden layers to be executed by each agent individually.
        :param n_s: number of MDP states per agent
        :param n_a: number of MDP actions per agent
        :param hidden_layers: list of ints that will determine the width of each hidden layer
        :param k: aggregation filter length
        :param ind_agg: before which MLP layer index to aggregate
        """
        super(Actor, self).__init__()
        self.k = k
        self.n_s = n_s
        self.n_a = n_a
        self.layers = [n_s] + hidden_layers + [n_a]
        self.n_layers = len(self.layers) - 1

        self.ind_agg = ind_agg  # before which conv layer the aggregation happens

        self.conv_layers = []

        for i in range(0, self.n_layers):

            if i == self.ind_agg:  # after the GSO, reduce dimensions
                step = k
            else:
                step = 1

            m = nn.Conv2d(in_channels=self.layers[i], out_channels=self.layers[i + 1], kernel_size=(step, 1),
                          stride=(step, 1))

            self.conv_layers.append(m)

        self.conv_layers = torch.nn.ModuleList(self.conv_layers)


    def forward(self, delay_state, delay_gso):
        """
        The policy relies on delayed information from neighbors. During training, the full history for k time steps is
        necessary.
        :param delay_state: History of states: x_t, x_t-1,...x_t-k+1 of shape (B,K,F,N)
        :param delay_gso: Delayed GSO: [I, A_t, A_t A_t-1, A_t ... A_t-k+1] of shape (B,K,N,N)
        :return:
        """
        batch_size = delay_state.shape[0]
        n_agents = delay_state.shape[3]
        assert delay_gso.shape[0] == batch_size
        assert delay_gso.shape[2] == n_agents
        assert delay_gso.shape[3] == n_agents

        assert delay_state.shape[1] == self.k
        assert delay_state.shape[2] == self.n_s
        assert delay_gso.shape[1] == self.k

        x = delay_state
        x = x.permute(0, 2, 1, 3)  # now (B,F,K,N)

        for i in range(self.n_layers):

            if i == self.ind_agg:  # aggregation only happens once - otherwise each agent operates independently
                x = x.permute(0, 2, 1, 3)  # now (B,K,F,N)
                x = torch.matmul(x, delay_gso)
                x = x.permute(0, 2, 1, 3)  # now (B,F,K,N)

            x = self.conv_layers[i](x)  # now (B,G,1,N)

            if i < self.n_layers - 1:  # last layer - no relu
                # x = self.layer_norms[i](x)
                x = torch.tanh(x)
                #x = F.relu(x)  # torch.tanh(x) #F.relu(x)
            # else:
            #     x = 10 * torch.tanh(x)

        x = x.view((batch_size, 1, self.n_a, n_agents))  # now size (B, 1, nA, N)

        # x = x.clamp(-10, 10)  # TODO these limits depend on the MDP

        return x

In [ ]:
# Define a state without delay
class MultiAgentStateWithoutDelay(object):

    def __init__(self, device, args, env_state, prev_state=None, k=None):
        """
        Create the state object that keeps track of the current state and GSO and history information
        :param device: CUDA device to use with PyTorch
        :param args:
        :param env_state:
        :param prev_state:
        """
        n_states = args.getint('n_states')
        n_agents = args.getint('n_agents')
        k = k or args.getint('k')
        # n_states = args.n_states
        # n_agents = args.n_agents
        # k = args.k

        # split up the state tuple
        state_value, state_network = env_state

        assert state_value.shape == (n_agents, n_states)
        assert state_network.shape == (n_agents, n_agents)
        assert np.sum(np.diag(state_network)) == 0  # assume no self loops

        # reshape values and network to correct shape
        state_value = state_value.transpose(1, 0)
        state_value = state_value.reshape((1, 1, n_states, n_agents))
        state_network = state_network.reshape((1, 1, n_agents, n_agents))

        # move matrices to GPU
        self.values = torch.Tensor(state_value).to(device)
        self.network = torch.Tensor(state_network).to(device)

        # compute current GSO - powers of the network matrix: current GSO: I, A_t, A_t^2... A_t^k-1
        self.curr_gso = torch.zeros((1, k, n_agents, n_agents)).to(device)
        self.curr_gso[0, 0, :, :] = torch.eye(n_agents).view((1, 1, n_agents, n_agents)).to(device)  # I
        for k_ind in range(1, k):
            self.curr_gso[0, k_ind, :, :] = torch.matmul(self.network, self.curr_gso[0, k_ind - 1, :, :])

        # delayed GSO: I, A_t-1, ...,  A_t-1 * ... * A_t-k
        self.delay_gso = torch.zeros((1, k, n_agents, n_agents)).to(device)
        self.delay_gso[0, 0, :, :] = torch.eye(n_agents).view((1, 1, n_agents, n_agents)).to(device)  # I
        if prev_state is not None and k > 1:
            self.delay_gso[0, 1:k, :, :] = torch.matmul(self.network, prev_state.delay_gso[0, 0:k - 1, :, :])

        # delayed x values x_t, x_t-1,..., x_t-k
        self.delay_state = torch.zeros((1, k, n_states, n_agents)).to(device)
        self.delay_state[0, 0, :, :] = self.values
        if prev_state is not None and k > 1:
            self.delay_state[0, 1:k, :, :] = prev_state.delay_state[0, 0:k - 1, :, :]


In [2]:
# Training functions from train.py
def run_experiment(args):
    # initialize gym env
    env_name = args.get('env')
    env = gym.make(env_name)

    if isinstance(env.env, gym_flock.envs.FlockingRelativeEnv):
        env.env.params_from_cfg(args)

    # use seed
    seed = args.getint('seed')
    env.seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # initialize params tuple
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    stats = train_cloning(env, args, device)

    return stats


In [ ]:
# main() equivalent from train.py
fname = 'multiagent_gnn_policies/cfg/cloning.cfg' #sys.argv[1]
config_file = path.join(path.abspath(''), fname) # For Jupyter
# config_file = path.join(path.dirname(__file__), fname)
config = configparser.ConfigParser()
config.read(config_file)

printed_header = False

if config.sections():
    for section_name in config.sections():
        if not printed_header:
            print(config[section_name].get('header'))
            printed_header = True

        stats = run_experiment(config[section_name])
        print(section_name + ", " + str(stats['mean']) + ", " + str(stats['std']))
else:
    val = run_experiment(config[config.default_section])
    print(val)

reward


In [13]:
config_file

'/Users/zz/Documents/GT20F/7000/multiagent_gnn_policies/cfg/cloning.cfg'

In [14]:
__file__

'/Users/zz/Documents/GT20F/7000/GNN_experiments'

In [16]:
path.dirname(__file__), fname

('/Users/zz/Documents/GT20F/7000', 'multiagent_gnn_policies/cfg/cloning.cfg')